In [ ]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
import json
import requests
from time import sleep
from tabulate import tabulate
from IPython.display import HTML, display, Image, FileLink, Markdown
import plotly.graph_objects as go
import kaleido
import os 

In [ ]:
%%appyter code_exec

{% set title = StringField(
    name='title',
    label='Notebook name',
    default='KEA3 Appyter',
    section="PRIMARY",
) %}

title = {{ title }}

In [ ]:
display(Markdown(f'# {title}'), display_id="title")

In [ ]:
%%appyter markdown

<center>
    <div style="font-size: 1.5rem; font-weight: 300">An appyter interface to the kinase enrichment analysis (KEA) database and enrichment analysis tool</div>
</center>

In [ ]:
def get_kea3_results(gene_set, query_name):
    ADDLIST_URL = 'https://maayanlab.cloud/kea3/api/enrich/'
    payload = {
        'gene_set': gene_set,
        'query_name': query_name
    }

    response = requests.post(ADDLIST_URL, data=json.dumps(payload))
    if not response.ok:
        raise Exception('Error analyzing gene list')
    sleep(1)

    return json.loads(response.text)

In [ ]:
def display_tables(lib, title):
    display(HTML(f'<h3>{title}</h3>'))
    for lib_name in lib:
        display(HTML(f'<h4>{lib_name}</h4>'))
        res_int = [[r['Rank'], r['TF'], f'{r["Intersect"]}/{r["Set length"]}', r['FET p-value'], r['FDR'], r['Odds Ratio'], f"{', '.join(r['Overlapping_Genes'].split(',')[0:10])}, ..."] for r in results[lib_name][:num_kinases]]
        display(HTML(tabulate(res_int, ['Rank', 'Kinase', 'Overlap', 'FET p-value', 'FDR', 'Odds Ratio', 'Overlapping Genes'], tablefmt='html')))
        tsv_name = f"{lib_name.replace(' ', '_')}.tsv"
        with open(tsv_name, 'w') as tsv_file:
            tsv_file.write(tabulate(res_int, ['Rank', 'Kinase', 'Overlap', 'FET p-value', 'FDR', 'Odds Ratio', 'Overlapping Genes'], tablefmt='tsv'))
        display(HTML(f'<a href="{tsv_name}">Download table in .tsv</a>'))

In [ ]:
%%appyter hide_code

{% do SectionField(name ='section', title = 'Perform Kinase Enrichment Analysis on a Gene/Protein List', 
                   subtitle = 'Kinase Enrichment Analysis 3 (KEA3) infers upstream kinases whose putative substrates are ' +
                               'overrepresented in a user-inputted list of genes or differentially phosphorylated proteins. ' +
                               'The KEA3 database contains putative kinase-substrate interactions collected from publicly ' +
                               'available datasets. Upload -OR- input a gene/protein list to retrieve the top associated kinases.', 
                   img = 'KEA3_transparent_logo.png')%}

## Get input list
Libraries are supersets of kinase substrate sets that are aggregated based on the database from which they are derived.

In [ ]:
%%appyter code_exec

gene_list_file = {{ FileField(
        name = 'Upload gene/protein list',
        label = 'Upload gene/protein list',
        default = '',
        examples = {'sample_gene_list.txt': 'https://raw.githubusercontent.com/MaayanLab/KEA3-Appyter/master/sample_gene_list.txt'},
        section = 'section'
    )}}

gene_list_input = {{ TextField(
        name = 'Input gene/protein list',
        label = 'Input gene/protein list (one per row)',
        default = 'ZNF264 \nTMPO \nISL2 \nMAP3K8 \nEFNB1 \nEIF3C \nOSBPL11 \nABCF1 \nUTRN \nOPRK1 \nTSC1 ' +
                    '\nGAB2 \nRPS3P2 \nDDX3X \nPPP1CA \nNF2 \nRBM3 \nIRAK1 \nKCNH2 \nNPR1 \nMOCOS \nITSN2 ' +
                    '\nMITF \nARAF \nDAPK2 \nEPHB2 \nCACNA1G \nYWHAZ \nGMFB',
        section = 'section',
    )}}


# Number of ranked kinases to display for the tables
num_kinases = {{ IntField(
        name = 'Input number of top kinases to display',
        label = 'Number of top kinases for tables',
        description = 'Input any integer from 1 to 100',
        default = 10,
        min = 1, 
        max = 100,
        section = 'section'
    )}}

In [ ]:
gene_input = None

# MeanRank threshold
threshold = 3

if gene_list_file:
    gene_list_file = open(gene_list_file,'r')
    gene_input = gene_list_file.read()
    gene_list_file.close()
elif gene_list_input:
    gene_input = gene_list_input
else:
    display(HTML('<h2>No kinase list was inputted or uploaded.</h2>'))

## Bar charts

In [ ]:
if gene_input:
    genes = [x.strip() for x in gene_input.split('\n')]
    results = get_kea3_results(genes, 'Query')
    k_libs_palette = {'BioGRID': 'rgb(196, 8, 8)', 'ChengKSIN': 'rgb(244, 109, 67)', 'ChengPPI': 'rgb(242, 172, 68)',
                      'HIPPIE': 'rgb(236, 252, 68)', 'mentha': 'rgb(165, 242, 162)', 'MINT': 'rgb(92, 217, 78)',
                      'PhosDAll': 'rgb(0, 138, 64)', 'prePPI': 'rgb(96, 191, 235)', 'PTMsigDB': 'rgb(14, 130, 201)',
                      'STRING': 'rgb(58, 50, 168)', 'STRING.bind': 'rgb(158, 50, 168)'}
    
    k_libs_means = {'STRING.bind': [0] * 10, 'ChengPPI': [0] * 10, 'PhosDAll': [0] * 10, 'BioGRID': [0] * 10,
                    'HIPPIE': [0] * 10, 'ChengKSIN': [0] * 10,
                    'STRING': [0] * 10, 'MINT': [0] * 10, 'mentha': [0] * 10, 'prePPI': [0] * 10,
                    'PTMsigDB': [0] * 10}

    libs_sorted = ['BioGRID', 'ChengKSIN', 'ChengPPI', 'HIPPIE', 'mentha', 'MINT', 'PhosDAll', 'prePPI', 'PTMsigDB', 'STRING', 'STRING.bind']
    
    results_mr_t = [r for r in results['Integrated--meanRank'] if len(r['Library'].split(';')) >= threshold][:num_kinases]
    # results_mr_t.sort(key=lambda kin: sum([int(k.split(',')[1]) for k in kin['Library'].split(';')]))
    results_mr_t = results_mr_t[::-1]
    sorted_kinases = [k['TF'] for k in results_mr_t[:num_kinases]]

    for i, kin in enumerate(results_mr_t[:num_kinases]):
        scores = kin['Library'].split(';')
        for s in scores:
            l, s = s.split(',')
            k_libs_means[l][i] = int(s)

    fig = go.Figure(data=[go.Bar(name=k_lib, x=k_libs_means[k_lib], y=sorted_kinases,
                                 marker=go.bar.Marker(color=k_libs_palette[k_lib]), orientation='h') for k_lib in
                          libs_sorted])

    fig.update_layout(barmode='stack')
    fig.update_layout(title={
        'text': 'Stacked Bar Chart of Sum of Ranks in Different Libraries',
        'y': 0.87,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top',
    },
        xaxis_title='Sum of Ranks in Different Libraries',
        yaxis_title='Kinases',
        font=dict(
            size=16,
            color='black'
        )
    )

    display(HTML(
        '<h2>This interactive bar chart summarizes the ranks of the top kinases from the different libraries.</h2>'))
    display(HTML(
        f'<i>Kinases are sorted by MeanRank. Only kinases with at least {threshold} contributed libraries are shown. ' +
        'To download the file as a PNG, click the camera button in the upper right corner of the chart.</i>'))
    fig.show()

## Tables

In [ ]:
if gene_input:
    display(HTML('<h2>The following tables display the integrated (summarizing) results of the top kinases.</h2>'))
    
    display(HTML('<h4>MeanRank</h4>'))
    res_mr = [[r['Rank'], r['TF'], r['Score'], r['Library'].replace(',', ': ').replace(';', ', '), f"{', '.join(r['Overlapping_Genes'].split(',')[0:10])}, ..."] for r in results['Integrated--meanRank'][:num_kinases]]
    display(HTML(tabulate(res_mr, ['Rank', 'Kinase', 'MeanRank', 'Libraries Ranks', 'Overlapping Genes'], tablefmt='html')))
    
    display(HTML('<h4>TopRank</h4>'))
    res_tr = [[r['Rank'], r['TF'], r['Score'], r['Library'].replace(',', ': ').replace(';', ', '), f"{', '.join(r['Overlapping_Genes'].split(',')[0:10])}, ..."] for r in results['Integrated--topRank'][:num_kinases]]
    display(HTML(tabulate(res_tr, ['Rank', 'Kinase', 'TopRank', 'Libraries Ranks','Overlapping Genes'], tablefmt='html')))

    display(HTML('<h2>The following tables display the rankings of the top kinases from each library.</h2>'))

    display_tables(['ChengKSIN', 'PTMsigDB', 'PhosDAll'], 'Kinase-substrate interaction')
    display_tables(['prePPI', 'BioGRID', 'mentha', 'MINT','HIPPIE', 'STRING.bind',  'ChengPPI'], 'Protein-protein interaction')
    display_tables(['STRING'], 'All associations')